In [1]:
!pip install selenium
!pip install webdriver_manager
!pip install -q jmd_imagescraper
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=1920x1080")

     |████████████████████████████████| 979 kB 5.2 MB/s 
     |████████████████████████████████| 359 kB 20.2 MB/s 
     |████████████████████████████████| 138 kB 53.1 MB/s 
     |████████████████████████████████| 4.0 MB 42.9 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 127 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing instal

# Scraping Description and URLs

In [2]:
#importing required libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd

In [3]:
#mounting google drive
from google.colab import drive 
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/MyDrive/DATA_690_NLP_Project_Recommender_System/Ujjwal/new_datasets/reviews_4.csv')

Mounted at /content/gdrive


In [5]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import plotly.express as px

#Grouping the dataset reviews on basis of name of the game
ProductReviewSummary = df.groupby("app_name")["clean_review_text"].apply(str)

p = ProductReviewSummary.to_frame()
p.reset_index(inplace=True)

#getting list of game ids in lit
lit = p.index.tolist()

In [ ]:
#designing a function that will scrape description and URLs with the help of "Selenium webdriver"
def get_desc_url(name):
    s = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    driver.get('https://store.steampowered.com/')
    
    
    driver.find_element_by_xpath('//*[@id="store_nav_search_term"]').send_keys(name)
    sleep(0.1)
    
    driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/div[1]/div[2]/div[4]/div/div/div/div[2]/div[2]/div/div[9]/div[1]/form/div/a/img').click()                          
    sleep(0.1)                    #/html/body/div[1]/div[7]/div[5]/div[1]/div[2]/div[4]/div/div/div/div[2]/div[2]/div/div[9]/div[1]/form/div/a/img
    
    driver.execute_script('document.cookie = \"birthtime=0; path=/; max-age=315360000\"')
    sleep(0.1)

    try:
         driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/form/div[1]/div/div[1]/div[3]/div[2]/div[3]/a[1]/div[2]/div[1]/span').click()
    except:
        sleep(0.1)

    try:
        driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/form/div[1]/div/div[1]/div[3]/div/div[3]/a[1]/div[2]/div[1]/span').click()
    except:
        sleep(0.1)

    gre = ''

    try:
         gre = driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/div[1]/div[3]/div[1]/div[3]/div[1]/div[2]/div[1]/div/div[2]').text
    except:
        sleep(0.1)

    try:
        gre = driver.find_element_by_xpath('/html/body/div[1]/div[7]/div[5]/div[1]/div[3]/div[1]/div[4]/div[1]/div[2]/div[1]/div/div[2]').text
    except:
        sleep(0.1)
        
    url = driver.current_url
        
    return gre, url

In [ ]:
#the scraped data will be collected into list which will later be converted into a dataset by tupling the relevant Descriptions, URLs and Game Names.
des = []
url = []
j=0
for i in lit:
      d, u = get_desc_url(i)
      des.append(d)
      url.append(u)
scrapedf = pd.DataFrame(list(zip(des, url, lit)), columns=['Description','URL', 'GameName'])
scrapedf.to_csv('Desc.csv')

# Scraping Images of each game

In [ ]:
#below code will scrape 3 images of each and every game present in the dataset and save those in appropriate folder name designated to game IDs
from pathlib import Path
root = Path().cwd()/"Images"
from jmd_imagescraper.core import *

for i in range(0, len(p)):
  duckduckgo_search(root, str(i), p['app_name'][i]+' game', max_results=3)

In [ ]:
#saving the images in drive
!cp -av '/content/Images' '/content/gdrive/MyDrive/690 Project/Sarang/Images' 